In [1]:
import pandas as pd
import numpy as np
import math

In [ ]:
# Load the dataset
df = pd.read_csv("500_Person_Gender_Height_Weight_Index.csv")

In [ ]:
# Assuming BMI is not provided in the dataset, calculate BMI
df['BMI'] = df['Weight'] / ((df['Height'] / 100) ** 2)

In [ ]:
def log_likelihood(feature, target, label):
    mean = df[df[target] == label][feature].mean()
    var = df[df[target] == label][feature].var()
    exponent = -(pow(df[feature] - mean, 2) / (2 * var))
    return -(math.log(2 * math.pi) / 2) - (math.log(math.sqrt(var))) - np.sum(exponent / 2)


In [ ]:
features = ['Height', 'Weight', 'BMI']
target = 'Gender'

In [ ]:
# Calculate log-likelihoods for male and female classes
like_male = sum(log_likelihood(f, target, 'Male') for f in features)
like_female = sum(log_likelihood(f, target, 'Female') for f in features)

print("Log-Likelihood for Male: ", like_male)
print("Log-Likelihood for Female: ", like_female)

In [ ]:
# Predict gender for a new data point
def predict_gender(height, weight, bmi, like_male, like_female):
    p_male = like_male + log_likelihood('Height', target, 'Male', height) + \
             log_likelihood('Weight', target, 'Male', weight) + \
             log_likelihood('BMI', target, 'Male', bmi)

    p_female = like_female + log_likelihood('Height', target, 'Female', height) + \
               log_likelihood('Weight', target, 'Female', weight) + \
               log_likelihood('BMI', target, 'Female', bmi)

    if p_male > p_female:
        return "Male"
    else:
        return "Female"


In [ ]:
# Test the prediction with a new data point
new_height = 170
new_weight = 70
new_bmi = new_weight / ((new_height / 100) ** 2)

prediction = predict_gender(new_height, new_weight, new_bmi, like_male, like_female)
print("Predicted Gender:", prediction)

In [ ]:
# Performance analysis
def performance_analysis(df, target):
    TP, FP, TN, FN = 0, 0, 0, 0
    for i in df.iterrows():
        p_male = like_male + log_likelihood('Height', target, 'Male', i[1]['Height']) + \
                 log_likelihood('Weight', target, 'Male', i[1]['Weight']) + \
                 log_likelihood('BMI', target, 'Male', i[1]['BMI'])

        p_female = like_female + log_likelihood('Height', target, 'Female', i[1]['Height']) + \
                   log_likelihood('Weight', target, 'Female', i[1]['Weight']) + \
                   log_likelihood('BMI', target, 'Female', i[1]['BMI'])

        if p_male > p_female:
            if i[1][target] == 'Male':
                TP += 1
            else:
                FP += 1
        else:
            if i[1][target] == 'Female':
                TN += 1
            else:
                FN += 1

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    accuracy = (TP + TN) / (TP + TN + FP + FN)

    return precision, recall, accuracy

precision, recall, accuracy = performance_analysis(df, target)
print("Precision: ", precision)
print("Recall: ", recall)
print("Accuracy: ", accuracy)